In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_pcb 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT
    business_unit,
    dc_id,
    product_id,
    pcb
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY business_unit, dc_id, product_id
               ORDER BY proc_date DESC, upload_time DESC
           ) AS rn
    FROM {catalog_name}.udp_wcm_silver_dataportal.d_cfg_pcb
    WHERE TRIM(UPPER(delete_indicator)) NOT IN ('YES', 'X') OR delete_indicator IS NULL
) AS sub
WHERE rn = 1
""")
